<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing II</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=463">MICRO-512</a>) taught by Dr. D. Sage, Dr. M. Liebling, Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL <mark>2024</mark>.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>,
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
     
</p>
<hr style="clear:both">
<h1>Lab 4.2: Feature detection</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: <mark>Thursday February 22, 2024</mark></p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red"><mark>Monday March 4, 2024</mark></span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=463">Moodle</a></p>
    <p style="margin:4px;"><b>Lab session</b>: <mark>Thursday 29 February in CM 1 2</mark></p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 6</p>
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## <a name="imports_"></a> Imports
In the next cell we import Python libraries we will use throughout the lab, as well as the `ImageViewer` class, created specifically for this course, which provides interactive image visualization based on the `ipywidgets` library:
* [`matplotlib.pyplot`](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.html), to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive,
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays,
* [`cv2`](https://docs.opencv.org/2.4/index.html), for image processing tasks.

We will then load the `ImageViewer` class (see the documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer) or run the Python command `help(viewer)` after loading the class).

Finally, we load the images you will use in the exercise to test your functions. 

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import warnings
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv 
from interactive_kit import imviewer as viewer 

# Load images to be used in this exercise
img_mouse     = cv.imread('images/mouse.tif',     cv.IMREAD_UNCHANGED)
img_fundus    = cv.imread('images/fundus.tif',    cv.IMREAD_UNCHANGED)
img_eiffel    = cv.imread('images/eiffel.tif',    cv.IMREAD_UNCHANGED)
img_bikesgray = cv.imread('images/bikesgray.tif', cv.IMREAD_UNCHANGED)
img_covid     = cv.imread('images/covid.tif',     cv.IMREAD_UNCHANGED)
img_nft       = cv.imread('images/nft.tif',       cv.IMREAD_UNCHANGED)
img_onion     = cv.imread('images/onion.tif',     cv.IMREAD_UNCHANGED)
img_grayfig   = cv.imread('images/grayfig.tif',   cv.IMREAD_UNCHANGED)
img_object    = cv.imread('images/objects.tif',   cv.IMREAD_UNCHANGED)

# Create some test images
dim = 200
szx = slice(int(0.475*dim), int(0.525*dim), None)
img_noise200 = 50.0 * np.random.rand(dim, dim) 

x, y = np.meshgrid(np.arange(-dim//2, dim//2), np.arange(-dim//2, dim//2))
img_circle200 = img_noise200 + (x**2 + y **2 <= (0.35*dim)**2)*255

img_cross200 = img_noise200 + np.zeros((dim, dim), dtype=np.float64)
img_cross200[szx,:] += 255.0
img_cross200[:,szx] += 255.0
img_cross200[szx,szx] -= 255.0

We also import the JavaScript `ImageAccess` class as `Image`. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/).

In [ ]:
%use javascript
// import image-access as Image
var Image = require('image-access')

# Feature detection (17 points)

In this lab we will compare different use cases of edge detection and ridge detection, presented in the Appendix of Chapter 6. These algorithms play an essential role in capturing key features and properties from an image when deep learning is not viable or simply not the easier solution. We will first implement the key parts of each to grasp how they work and then explore some of the tricks that OpenCV has reserved for us behind the scene.

As customary for edge detection we will work with greyscale images in this lab. This is to minimize the computational cost and simplify the complexity of edge detection algorithms. 

## <a id="ToC_2_Feature_detection"></a>Table of contents
1. [Original Canny edge detector](#1.-Original-Canny-edge-detector-(17-points)) 
    1. [Noise reduction](#1.A.-Noise-reduction-(1-points)) **(1 points)**    
    2. [Magnitude and phase of the gradient](#1.B.-Magnitude-and-phase-of-the-gradient-(3-points)) **(3 points)** 
    3. [Non-maximum suppression](#1.C.-Non-maximum-suppression-(3-points)) **(3 points)** 
    4. [Hysteresis thresholding](#1.D.-Hysteresis-thresholding-(4-points)) **(5 points)**
    5. [Comparison with OpenCV](#1.E.-Comparison-with-OpenCV)
    6. [The switch to OpenCV](#1.F.-The-switch-to-OpenCV-(2-points)) **(2 points)**
2. [Original ridge detector](#2.-Original-ridge-detector-(3-points)) 
    1. [Motivation](#2.A.-Motivation)
    2. [Implementation](#2.B.-Implementation-(3-points)) **(3 points)**

### Visualize images
First of all, run the cell below to get familiar with the images we will be using. Remember you can use the `Widgets` to cycle through the images.

In [ ]:
%use sos
# Display all the images we will use in this lab
images_list = [img_object, img_mouse, img_fundus, img_eiffel, img_bikesgray, img_covid, img_nft, img_onion, img_cross200, img_circle200, img_grayfig]
plt.close('all')
imgs_viewer = viewer(images_list, widgets=True, use_slider=False)

# 1. Original Canny edge detector (13 points)
[Back to table of contents](#ToC_2_Feature_detection)

Over time, detecting edges has proven to be a great way to capture important features from images. They could be indicative of discontinuities in the environment, depth or brightness changes, varying material properties etc. There are drawbacks to the method, for example, a shadow casted on a flat surface would result in an edge being detected... Nevertheless, the commonly used Canny edge detector remains a fundamental tool in image processing.

The algorithm consists of four parts that we will now cover in detail.
1. Noise reduction
2. Gradient magnitude and phase calculation 
3. Non-maximum suppression
4. Hysteresis thresholding

<center><img src="images/edge_detector_schematic.png" width="800"/></center>

As it turns out we have in our lab a copy of John F. Canny's thesis.

<center><img src="images/thebook.jpg" width="800"/></center>

In this first part we will be using the following two synthetic images `img_cross200` and `img_circle200`. Run the next cell and observe how the noise could make it tricky to distinguish the shapes' edges.

In [ ]:
%use sos
# Display the two images
image_list = [img_cross200, img_circle200]
title_list = ['Cross200 Original', 'Circle200 Original']
plt.close('all')
view1 = viewer(image_list, title=title_list, subplots=(1,2), joint_zoom=True)

## 1.A. Noise reduction (1 points)
[Back to table of contents](#ToC_2_Feature_detection)

In this step, the goal is to mask any undesired features of lesser importance to our image analysis. Imagine an older low quality image or one that has been taken through a dirty window - you don't want to see that mess, do you? From a numerical point of view it is also useful when dealing with synthetic images, that do not have smooth transitions, whose uniformity could be hurtful to the next steps of the algorithm.

For this step we will use the gaussian filter you implemented in the warm up notebook of this lab. We will spare you the details this time and make use of OpenCV directly. In the next cell, **for 1 point**, complete the `smooth_gaussian` function so as to obtain a **blurred image** using [`cv.GaussianBlur`](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1). If you are unsure about its input parameters, go check the documentation. We will provide `sigma`, so make sure to set the kernel size `ksize` to be computed from sigma, and set the `borderType` parameter of `cv.GaussianBlur` to the `borderType` input parameter. Use the following cell to check your output.

In [ ]:
%use sos
# Function that implements a Gaussian smoothing using cv.GaussianBlur
def smooth_gaussian(image, sigma=3, borderType=cv.BORDER_REFLECT):
    # convert image to high precision floating point values
    image = np.float64(image)
    blurred_image = np.zeros(image.shape)
    
    # YOUR CODE HERE
    
    return blurred_image

In [ ]:
%use sos
# Apply the Gaussian blurring to the two test images
img_cross200_blur = smooth_gaussian(img_cross200)
img_circle200_blur = smooth_gaussian(img_circle200)
# Display the resulting images together with their originals
image_list = [img_cross200, img_circle200, img_cross200_blur, img_circle200_blur]
title_list = ['Cross200 Original', 'Circle200 Original', 'Cross200 Blurred', 'Circle200 Blurred']
plt.close('all')
view1a = viewer(image_list, title=title_list, subplots=(2,2), joint_zoom=True) 

Let's perform a quick sanity check. We should have a lower variance in the blurred image.

In [ ]:
%use sos
# Basic sanity check
# Lower variance / standard deviation in the blurred image
if np.std(img_cross200) < np.std(img_cross200_blur):
    print('WARNING!\nThis does not seem right, make sure you called the function using the correct parameters.')
else :
    print('Congratulations! Your blurring passed the sanity check.\nRemember that this is not a guarantee that everything is correct.')

## 1.B. Magnitude and phase of the gradient (3 points)
[Back to table of contents](#ToC_2_Feature_detection)

In this step, the goal is to extract the edge features from our image. An edge is characterized by an abrupt change in the image magnitude and the next steps of the algorithm care about the direction of the gradient which we call phase. 

For this step we will use the OpenCV Sobel filter [`cv.Sobel`](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gacea54f142e81b6758cb6f375ce782c8d) to compute our gradients. If you are unsure about its input parameters, go check the documentation. In the next cell, **for 3 points**, complete the `compute_gradient_features` function so as to compute the **magnitude, phase and normalized gradients along x and y**. We will provide `ksize` and `borderType` set by default to `BORDER_REFLECT`. Use the following cell to check your output.

<div class="alert alert-info">
<b>Hints:</b> 
<ul>
    <li>Take a moment to go thoroughly through the documentation! It will make your life much easier.</li>
    <li><code>cv.Sobel</code>: set <code>ddepth=cv.CV_64F</code> since the gradient can be a negative floating point number.</li>
    <li>Definitions, where $\partial_{x,y} I(x,y) = \frac{\partial I(x,y)}{\partial x,y}$ and $I(x,y)$ being the input image:
    <ul>
        <li>magnitude: $|| \nabla I(x,y)|| = \sqrt{(\partial_{x} I)^2 + (\partial_{y} I)^2}$</li> 
        <li>phase: $ \angle\big( \nabla I(x,y)\big) = \arctan\big(\partial_{y} I / \partial_{x} I\big)$.</li>
        <li>normalization: $\partial_{x,y} I_{\scriptsize{\mbox{norm}}}(x,y) = \frac{\partial_{x,y} I(x,y)}{|| \nabla I(x,y)||}$.</li>
    </ul>
    <li><code><a href="https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html" style="color:black">np.arctan2</a></code>: could help you get the correct angle in all four quadrants.</li>
    <li>When normalizing the $x$ and $y$ components of the gradient, prevent any division by zero.</li>
    <li>To ensure that you set the correct input parameter in a function call always specify the name of the input parameter you want to set.</li>
</ul>
</div>

In [ ]:
%use sos
# Function that computes the magnitude, phase, as well as the x- and y-components of the gradient
def compute_gradient_features(image, ksize=3, borderType=cv.BORDER_REFLECT):
    magnitude = np.zeros(image.shape)
    phase = np.zeros(image.shape)
    norm_gradX = np.zeros(image.shape)
    norm_gradY = np.zeros(image.shape)
    
    # Calculate the x and y components of the gradient cv.Sobel
    # YOUR CODE HERE
    
    # Calculate the magnitude and phase of the gradient
    # YOUR CODE HERE
    
    # Normalize the x and y components of the gradient by the magnitude of the gradient
    # YOUR CODE HERE
    
    return magnitude, phase, norm_gradX, norm_gradY

In [ ]:
%use sos
# Compute the gradient features on the cross and circle images
img_cross200_magphase = np.array(compute_gradient_features(img_cross200_blur))
img_circle200_magphase = np.array(compute_gradient_features(img_circle200_blur))
# Prepare image and title lists for the visualization
image_list = [img_cross200_blur, img_circle200_blur, img_cross200_magphase[0], img_circle200_magphase[0], 
                                                     img_cross200_magphase[1], img_circle200_magphase[1],
                                                     img_cross200_magphase[2], img_circle200_magphase[2],
                                                     img_cross200_magphase[3], img_circle200_magphase[3]]
title_list = ['Cross200 Blurred', 'Circle200 Blurred', 'Cross200 Magnitude',  'Circle200 Magnitude',
                                                       'Cross200 Phase',      'Circle200 Phase',
                                                       'Cross200 Gradient X', 'Circle200 Gradient X',
                                                       'Cross200 Gradient Y', 'Circle200 Gradient Y']
plt.close('all')
view1b = viewer(image_list, title=title_list, subplots=(5,2), joint_zoom=True, widgets=True)

Let's perform a quick sanity check. The magnitude values should be positive and only the edges of the shapes should have high intensity values. The phase values should span the entire $[-\pi, \pi]$ range. The normalised x- and y-components of the gradient should be in the range $[-1,1]$. 

In [ ]:
%use sos
# Basic sanity checks
error_check = False
# Magnitude values expected to be positive
if not (np.all(img_cross200_magphase[0] >= 0) and np.all(img_circle200_magphase[0] >= 0)) :
    print('WARNING!\nYour magnitude values should all be positive.')
    error_check = True

# The inner cross should now look darker
if not (np.mean(img_cross200_magphase[0][dim//2,:] + img_cross200_magphase[0][:,dim//2]) < 0.25*np.mean(img_cross200[dim//2,:] + img_cross200[:,dim//2]) and \
        np.mean(img_circle200_magphase[0][dim//2,:] + img_circle200_magphase[0][:,dim//2]) < 0.25*np.mean(img_circle200[dim//2,:] + img_circle200[:,dim//2])):
    print('WARNING!\nOnly the edges of the shapes should have high intensity values, the inside should be dark.')
    error_check = True

# Phase values span the [-pi, pi] range
if np.abs(np.max(img_circle200_magphase[1]) - np.pi) > 1e-3 or np.abs(np.min(img_circle200_magphase[1]) + np.pi) > 1e-3 or\
   np.abs(np.max(img_cross200_magphase[1]) - np.pi) > 1e-3 or np.abs(np.min(img_cross200_magphase[1]) + np.pi) > 1e-3:
    print('WARNING!\nYour phase values should span the range [-pi, pi].')
    error_check = True
    
# Normalised gradient values are within [-1, 1]
if np.abs(np.max(img_circle200_magphase[2]) - 1) > 1e-3 or np.abs(np.min(img_circle200_magphase[2]) + 1) > 1e-3 or\
   np.abs(np.max(img_circle200_magphase[3]) - 1) > 1e-3 or np.abs(np.min(img_circle200_magphase[3]) + 1) > 1e-3:
    print('WARNING!\nYour normalised x- and y-components of the gradient should be in the range [-1, 1].') 
    error_check = True
    
if error_check:
    print('Make use of the above visualization to locate your errors')
else :
    print('Congratulations! Your gradient features passed the sanity check.\nRemember that this is not a guarantee that everything is correct.')

## 1.C. Non-maximum suppression (3 points)
[Back to table of contents](#ToC_2_Feature_detection)

In this step, the goal is to filter our magnitude image and eliminate the pixels that do not represent a peak along the direction of the gradient, akin to determining a zero crossing of the second order (directional) derivative.

For this step we will ask you to **code in JavaScript** to get a lower level understanding of how this works. In the second next cell, **for 2 points**, your task is to implement `applyNonMaximumSuppressionInterpolation` where you will examine whether the **gradient magnitude** pixel value is **strictly greater** than its two neighbours **along the gradient direction**, in which case we want to keep it, otherwise we will discard it.

The function and parameters are given as follows:

<code>applyNonMaximumSuppressionInterpolation(magnitude, norm_grad_x, norm_grad_y)</code>
<ul>
    <ul>
        <li><code>magnitude</code> is the gradient magnitude</li>
        <li><code>norm_grad_x</code> and <code>norm_grad_y</code> are the normalised x- and y-components of the gradient </li>
    </ul>
</ul>

The output shall be an image where all **non-maximum pixels are suppressed** (set to zero).

<div class="alert alert-info">
<b>Hints:</b> 
<ul>
    <li>The gradient direction is perpendicular to the edges and is given by the vector $\boldsymbol{u} = \nabla I(\boldsymbol{x})$, with $||\boldsymbol{u}|| = 1$.</li>
    <li>The mathematical expression for NMS (non-maximum suppression) is to keep only pixels for which $|| \nabla I(\boldsymbol{x})|| > || \nabla I(\boldsymbol{x} \pm \boldsymbol{u})||$</li>
</ul>
</div>

Since the location $\boldsymbol{x} \pm \boldsymbol{u}$ that we want to check will not always correspond to an exact integer value, we need to interpolate the pixel values for the given location. In the next cell we provide you the function `getInterpolatedPixel(image, x, y)` that returns the in-between pixel values determined using [bilinear interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation). You will cover interpolation in a future lab, in much greater detail.

The function `getInterpolatedPixel(image, x, y)` takes as input parameters:
* `image` : The image from which we want to get an interpolated pixel value,
* `x` : The x location, which we want to interpolate and
* `y` : The y location, which we want to interpolate.

And returns the interpolated pixel value of `image` at (`x`, `y`).

In [ ]:
%use javascript
// Simple linear interpolation function
function getInterpolatedPixel(image, x, y){
    return (x - parseInt(x)) * (image.getPixel(parseInt(x)+1, parseInt(y)+1) * (y - parseInt(y)) - image.getPixel(parseInt(x)+1, parseInt(y)) * ((y - parseInt(y)) - 1.0)) - ((x - parseInt(x)) - 1.0) * (image.getPixel(parseInt(x), parseInt(y)+1) * (y - parseInt(y)) - image.getPixel(parseInt(x), parseInt(y)) * ((y - parseInt(y)) - 1.0));
}

**Note:** You don't need to worry about the boundary conditions here, because `.getPixel` already applies mirror boundary conditions by default. You will see later that OpenCV deals with them differently and what effects that has.

In [ ]:
%use javascript
// Function that applies non-maximum suppresion using bilinear interpolation
function applyNonMaximumSuppressionInterpolation(magnitude, norm_grad_x, norm_grad_y){
    var nms = new Image(magnitude.shape());
    
    // YOUR CODE HERE
    
    return nms;
}

Run the following two cells to visualize your output. Your edges should be comprised of a single-pixel wide line.

In [ ]:
%use javascript
%get img_cross200_magphase img_circle200_magphase
%put img_cross200_nms img_circle200_nms

// Apply the nms to the two test images
var img_cross200_nms = applyNonMaximumSuppressionInterpolation(
    new Image(img_cross200_magphase[0]), new Image(img_cross200_magphase[2]), new Image(img_cross200_magphase[3]) 
).toArray()
var img_circle200_nms = applyNonMaximumSuppressionInterpolation(
    new Image(img_circle200_magphase[0]), new Image(img_circle200_magphase[2]), new Image(img_circle200_magphase[3]) 
).toArray()

In [ ]:
%use sos
# Convert JavaScript images to numpy arrays
img_cross200_nms = np.array(img_cross200_nms)
img_circle200_nms = np.array(img_circle200_nms)
# Prepare image and title lists for visualization
image_list = [img_cross200_magphase[0], img_circle200_magphase[0], img_cross200_nms, img_circle200_nms]
title_list = ['Cross200 Magnitude', 'Circle200 Magnitude', 'Cross200 NMS', 'Circle200 NMS']
plt.close('all')
view1c = viewer(image_list, title=title_list, subplots=(2,2), joint_zoom=True, widgets=True)

Let's perform a quick sanity check and see if we can locate the maxima in the gradient features of a $7 \times 7$ **impulse image** computed using `ksize=5`. The 8 pixels around middle one should be highlighted. Run the following three cells to create the test image, apply your nms function to it and perform a small sanity check.

In [ ]:
%use sos
# Create an impulse image
impulse = np.zeros((7,7))
impulse[3,3] = 1
# Compute the gradient features of the impulse image
impulse_magphase = np.array(compute_gradient_features(impulse, ksize=5))

In [ ]:
%use javascript
%get impulse_magphase
%put impulse_nms

// Apply the nms to the impulse images' gradient features
var impulse_nms = applyNonMaximumSuppressionInterpolation(
    new Image(impulse_magphase[0]), new Image(impulse_magphase[2]), new Image(impulse_magphase[3]) 
).toArray()

In [ ]:
%use sos
# Convert JavaScript list to numpy array
impulse_nms = np.array(impulse_nms)
# Prepare image and title lists for visualization
image_list = [impulse, impulse_magphase[0], impulse_nms]
title_list = ['7x7 Impulse', 'Magnitude', 'NMS with Interpolation']
plt.close('all')
view = viewer(image_list, title=title_list, subplots=(1,3), joint_zoom=True, widgets=True)
# Sanity check    
if int(np.sum(impulse_nms)) != 93:
    print('WARNING!\nYour function did not suppress the right pixels or you modified their values.')
else:
    print('Congratulations! Your non-maximum suppression passed the sanity check.\nRemember that this is not a guarantee that everything is correct.')

### Multiple Choice Question

* Q1: Now that you have passed the sanity check, can you tell why the lines in the Cross200 image are not exacly straight?

    1. The gradients are not accurate enough to properly distinguish the interpolated pixels.
    2. Because the for loops explore the image pixel by pixel, when two maxima are close, one gets chosen because it came first.
    3. We forgot to normalize the image before converting its pixel values to floating-point numbers.
    4. It is caused by the noise and limited blurring effect.
    
In the next cell, **for $1$ point**, modify the variable `answer_one` to reflect your answer. The following cell is for you to check that your answer is in the valid range.

In [ ]:
%use sos
# Modify this variable to reflect your answer
answer_one = None

# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer_one in [1, 2, 3, 4]:
    print('WARNING!\nChoose one of 1, 2, 3 or 4.')

<div class='alert alert-info'>
    <b>Note:</b> The idea presented here can be generalised to filter out neural network outputs for object detection. A typical task for these nets is to place bounding boxes around the objects it detects. However, there are many ways one could place such boxes. To select one that fits best, a variant of non-maximum suppresion is applied.
</div>

## 1.D. Hysteresis thresholding (4 points)
[Back to table of contents](#ToC_2_Feature_detection)

In this step, the goal is to further narrow down the pixels we would like to keep as edges. The idea is to manually tune two thresholds, $T_{low}$ and $T_{high}$, that divide our pixels into three categories: `'strong'` edges, `'weak'` edges, and `'background'`. Starting from the `'strong'` edges we will track and keep the `'weak'` edges that are connected to them and discard the remaining ones as being part of the `'background'`. 

<ul>
    <li><code>'strong'</code>: pixels we are sure we'd like to keep right off the bat</li>
    <li><code>'weak'</code>: uncertain at first, we'll keep them if they are connected to a strong one</li>
    <li><code>'background'</code>: we know we don't care about these</li>
</ul>

For this step we will ask you to **code in JavaScript** the different subfunctions of `thresholdEdgesHysteresis(image, tl, th)`, to make sure that you get a lower level understanding of how this works. First of all, run the next cell to declare the function (and follow the transformation process of the local variable `array`!). 

In [ ]:
%use javascript
function thresholdEdgesHysteresis(image, tl, th) {
    var output = new Image(image.shape());
    var array = new Image(image.shape());
    // initialise array with semantic edge labels 
    array = labelImageEdges(image, tl, th);
    
    // track connected edges starting from strong pixels
    array = trackAllEdges(array);
    
    // keep 'strong' edges, discard remaining 'weak' edges as 'background'
    array = thresholdEdges(array);
    
    return array
}

If you read the names of the $3$ functions that you will implement in the cell above, you might already have gotten an idea of the algorithm we want you to implement:
<ol>
    <li> Assign a semantic value to each pixel:
        <ul>
            <li><code>'background'</code> if value $\le T_{low}$</li>
            <li><code>'strong'</code> if value $>T_{high}$
            <li><code>'weak'</code> otherwise</li>
        </ul>
    </li> 
    <li> For each <code>'strong'</code> pixel, <b>recursively</b> track the <code>'weak'</code> edges that are <a href="https://en.wikipedia.org/wiki/Pixel_connectivity#8-connected"><b>8-connected</b></a> to it and make those pixels <code>'strong'</code> as well. i.e. for each <code>'strong'</code> pixel, call your recursive tracking function (<a href="https://en.wikipedia.org/wiki/Recursion_(computer_science)">a little curiosity on recursivity</a>).</li>
    <li> Keep <code>'strong'</code> pixels only (set them to <b>255</b>) and discard the remaining <code>'weak'</code> edges as <code>'background'</code> (set them to <b>0</b>).
</ol>

Following is the picture taken from your course slides that might help.

[<center><img src="images/edgetrack.png" width="800"/></center>](images/edgetrack.png)

<div class='alert alert-warning'>
    <b>Technical note on the use of dictionaries:</b> <a href = "https://en.wikibooks.org/wiki/A-level_Computing/AQA/Paper_1/Fundamentals_of_data_structures/Dictionaries">dictionaries</a>, a fundamental data structure also called a hashmap. In the next cell we provide a <b>dictionary</b> called <code>edge</code> where the <b>key</b> (of type <code>string</code>) maps to the <b>value</b> (of type <code>int</code>). We can simply access it with <code>edge[key]</code> to get the <code>value</code>. As you can see from the code, we define <code>'strong'</code> as <code>-1</code>, <code>'weak'</code> as <code>-2</code> and <code>'background'</code> as <code>-3</code>. We invite you to look up the documentation, should you need it, and learn how to use it - for your own good. You might find Python has a few cool dictionary tricks that allow for very compact and efficient code. Fortunately, the syntax in Python and Javascript is identical, so we can use the basics seamlessly.
</div>

In the next cell, **for 1 point**, complete the function `labelImageEdges(array, tl, th)` to implement the first step in the above algorithm description. Then run the cell below that for a quick sanity check. In it, we will define the image `rocky_path`, which is a black-and-white version of the one shown in the course. Through the rest of the section we will track its transformation through the process. If you have any doubt about the algorithm, make sure to understand the answers at every step of the way!  

In [ ]:
%use javascript
// semantic edge pixel dictionary, mapping labels to numerical values (do not change)
var edge = {
    'strong':     -1,
    'weak':       -2,
    'background': -3,
};

function labelImageEdges(array, tl, th){
    for(var i = 0; i < array.nx; i++){
        for(var j = 0; j < array.ny; j++){
            
            // YOUR CODE HERE
            
        }
    }
    return array
}

In [ ]:
%use javascript
%put rocky_path 
%put rocky_path_lab
%put rocky_path_lab_corr
// create the test image and it's labelled version
rocky_path = [
    [0, 9, 0, 0, 0, 0, 5],
    [0, 0, 9, 9, 0, 0, 0],
    [0, 0, 0, 0, 5, 0, 0],
    [5, 5, 0, 0, 9, 0, 0],
    [0, 5, 0, 0, 5, 5, 0],
    [0, 0, 0, 5, 0, 9, 0],]
rocky_path_lab_corr = [
    [-3, -1, -3, -3, -3, -3, -2],
    [-3, -3, -1, -1, -3, -3, -3],
    [-3, -3, -3, -3, -2, -3, -3],
    [-2, -2, -3, -3, -1, -3, -3],
    [-3, -2, -3, -3, -2, -2, -3],
    [-3, -3, -3, -2, -3, -1, -3],]

// perform image labelling on the test image
var rocky_path_lab = labelImageEdges(new Image(rocky_path), 3, 7).toArray();
if(new Image(rocky_path_lab_corr).imageCompare(new Image(rocky_path_lab)) == false) {    
    console.log("WARNING!!\nYour image labeling is not quite correct yet")
} else {    
    console.log("Your image labeling seems to be correct!")
}

In the next cell, **for 1 point**, you will implement the tracking process. Complete the functions `trackEdge(array, x, y)`, which begins to track an edge from the location `[x, y]`. The function and parameters are defined as:

<ul>
    <li><code>array</code> : an image of labelled pixels. Each pixel has either value <code>-1</code> for <code>'strong'</code>, <code>-2</code> for <code>'weak'</code> or <code>-3</code> for <code>'background'</code></li>
    <li><code>x</code> and <code>y</code> the coordinates of the pixel, from where you will start / continue the edge tracking</li>
</ul>

The function does not return any value, instead it directly modifies the input image `array`. Note that, because of the recursivity of the function, the initialization pixel could be any `'strong'` edge, and the result would not change. For example, in the `rocky_path_image`, initialization on locations `[0, 1]` and `[3, 4]` would both give the same result. Here, we have already coded `trackAllEdges`, a function that takes a labelled array as input and iterates every edge, tracking the ones that were labelled as `'strong'`.

<div class='alert alert-info'>
    <b>Hints:</b>
    <ul>
        <li>Since <code>trackEdge</code> is a recursive function, you will need to call <code>trackEdge</code> from withing <code>trackEdge</code>.</li>
    <li>You should not track any edges beyond the boundaries of the image. Depending on your boundary conditions, if not careful, your <code>trackEdge</code> function could start bouncing back and forth between two boundaries of the image, creating an infinite loop.
    </ul>
</div>

Complete the function in the following cell.

In [ ]:
%use javascript

function trackAllEdges(array){
    for(var x = 0; x < array.nx; x++) {
        for(var y = 0; y < array.ny; y++) {
            if(array.getPixel(x, y) == edge['strong']) {
                trackEdge(array, x, y);
            }
        }
    }
    return array
}

function trackEdge(array, x, y) { // separate this in separate cell, with its sanity check
    // loop over 8-connected neighbourhood of pixel (x,y) in array and track its connected edges
    for(var xi = x-1; xi <= x+1; xi++) {
        for(var yj = y-1; yj <= y+1; yj++) {
            
            // YOUR CODE HERE
            
        }
    }    
    return
}

As a sanity, let's apply the `trackEdge` function on the `epfl_logo` test image, all the EPFL pixels should be EPFL-RED, except for the top row. Run the next three cells to visualize the result.

In [ ]:
%use sos
# Basic sanity check
epfl_logo = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
])
epfl_logo[epfl_logo == 0] = -3
epfl_logo[epfl_logo == 1] = -2
epfl_logo[-1, -1] = -1

In [ ]:
%use javascript
%get epfl_logo
%put epfl_logo_track

// create test image (epfl_logo)
var epfl_logo_track = new Image(epfl_logo);
// track the edge recursively using trackEdge
trackEdge(epfl_logo_track, epfl_logo_track.nx-1, epfl_logo_track.ny-1);
// convert image to array for use in Python
epfl_logo_track = epfl_logo_track.toArray();

In [ ]:
%use sos
# Convert the resulting image to numpy
epfl_logo_track = np.array(epfl_logo_track)
# Visualize the original image and the tracked edge
image_list = [epfl_logo, epfl_logo_track]
title_list = ['EPFL Logo', 'EPFL Logo after EdgeTrack']
plt.close('all')
view = viewer(image_list, title=title_list, subplots=(1,2), joint_zoom=True, widgets=True, pixel_grid=True, cmap = 'Reds', clip_range = [-3, 0])

if np.any(epfl_logo_track[1:,:] == -2):
    print('WARNING!\nYour edge tracking function did not detect all of the pixels recursively 8-connected to the bottom right pixel.\n')
else:
    print('Congratulations! Your edge tracking function passed the sanity check.\nRemember that this is not a guarantee that everything is correct.') 

Now, let's get to the final step of the algorithm, the thresholding. **For 1 point**, complete the function `thresholdEdges(array)` in the next cell, that takes as input a previously labelled and tracked image, and returns a binary image with **values $\{0, 255\}$ for non-edges and edges** respectively. Then, run the cell after that for a final sanity check on the `rocky_path` image. 

In [ ]:
%use javascript

function thresholdEdges(array){
    for(var i = 0; i < array.nx; i++){
        for(var j = 0; j < array.ny; j++){
            
            // YOUR CODE HERE
            
        }
    }
    return array
}

In [ ]:
%use javascript
%put rocky_path_tracked
%put rocky_path_tracked_corr
%put rocky_path_thr_corr
%put rocky_path_thr
// create the test image and it's labelled version
rocky_path_tracked_corr = [
    [-3, -1, -3, -3, -3, -3, -2],
    [-3, -3, -1, -1, -3, -3, -3],
    [-3, -3, -3, -3, -1, -3, -3],
    [-2, -2, -3, -3, -1, -3, -3],
    [-3, -2, -3, -3, -1, -1, -3],
    [-3, -3, -3, -1, -3, -1, -3],]

rocky_path_thr_corr = [
    [0, 255,  0,  0, 0, 0, 0],
    [0, 0, 255, 255, 0, 0, 0],
    [0, 0, 0,   0, 255, 0, 0],
    [0, 0, 0,   0, 255, 0, 0],
    [0, 0, 0, 0, 255, 255, 0],
    [0, 0, 0, 255, 0, 255, 0],]

// perform hysteresis edge tracking on the labelled test image. For completeness, we also extract the tracked version
var rocky_path_tracked = trackAllEdges(new Image(rocky_path_lab_corr)).toArray();
var rocky_path_thr = thresholdEdges(new Image(rocky_path_tracked_corr)).toArray();
if(new Image(rocky_path_thr_corr).imageCompare(new Image(rocky_path_thr)) == false) {    
    console.log("WARNING!!\nYour thresholding is not quite correct yet.")
} else {    
    console.log("Congrats! Your thresholding seems to be correct!")
}


Nice! You have now implemented the three steps in hystheresis thresholding. Let's now look at the bigger picture and re-run the function `thresholdEdgesHysteresis`. Run the next cell to compute the final output. Then run the Python cell after that to visualize the results.

In [ ]:
%use javascript
%put rocky_path_tlh

// perform hysteresis edge tracking on the test image
var rocky_path_tlh = thresholdEdgesHysteresis(new Image(rocky_path), 3, 7).toArray();
if(new Image(rocky_path_thr_corr).imageCompare(new Image(rocky_path_tlh)) == false) {    
    console.log("WARNING!!\nYour hystheresis thresholding is not quite correct yet. Look at the feedback from the indvidual functions for detail!")
} else {    
    console.log("Congrats! Your hystheresis thresholding workflow seems to be perfect!")
}

In [ ]:
%use sos
# Convert initial JS arrays to numpy
rocky_path = np.array(rocky_path)
rocky_path_tlh = np.array(rocky_path_tlh)
rocky_path_thr_correct = np.array(rocky_path_thr_corr)

if np.allclose(rocky_path_tlh, rocky_path_thr_correct):
    print('Congrats! You seemed to have mastered hystheresis thresholding. Take a look at the replication of the course image below.') 
    # Visualize the original and result image
    image_list = [rocky_path, rocky_path_tlh]
    title_list = ['Rocky Path', 'Rocky Path after Hysteresis']
    plt.close('all')
    view = viewer(image_list, title=title_list, subplots=(1,2), joint_zoom=True, widgets=True, pixel_grid=True)
else :
    print('WARNING!!!\nSomething s not quite correct yet. You have already gotten some feedback before, but take a look at the following viewer with the \
          the correct steps of the process in comparison to your implementations. Remember that each of the steps is evaluated separately, with the correct input.') 
    rocky_path_lab_correct = np.array(rocky_path_lab_corr)
    rocky_path_lab = np.array(rocky_path_lab)
    rocky_path_tracked_correct = np.array(rocky_path_tracked_corr)
    rocky_path_tracked = np.array(rocky_path_tracked)
    rocky_path_thr = np.array(rocky_path_thr)
    image_list = [rocky_path_lab_correct, rocky_path_tracked_correct, rocky_path_thr_correct, rocky_path_lab, rocky_path_tracked, rocky_path_thr]
    title_list = ['GT Labeling', 'GT Edge Tracking', 'GT Binarization', 'Labeling', 'Edge Tracking', 'Binarization']
    view = viewer(image_list, title=title_list, subplots=(2,3), joint_zoom=True, widgets=True, pixel_grid=True)
    


Now that you have passed all sanity checks, let's take a look at the output of our final edge detection stage on Cross200 and Circle200. Run the next two cells and observe the output edges. Go back to your implementation if it doesn't feel quite right.

In [ ]:
%use javascript
%get img_cross200_nms img_circle200_nms 
%put img_cross200_tlh img_circle200_tlh tl th
// set the thresholds
var tl = 50;
var th = 100;
// apply the hysteresis edge tracking to the non-maximum suppresion images of the two test images
var img_cross200_tlh = thresholdEdgesHysteresis(new Image(img_cross200_nms), tl, th).toArray();
var img_circle200_tlh = thresholdEdgesHysteresis(new Image(img_circle200_nms), tl, th).toArray();

In [ ]:
%use sos
# Convert JS arrays to numpy
img_cross200_tlh = np.array(img_cross200_tlh)
img_circle200_tlh = np.array(img_circle200_tlh)
# Display the two test images and their edges
image_list = [img_cross200, img_circle200, img_cross200_tlh, img_circle200_tlh]
title_list = ['Cross200', 'Circle200', f'Cross200 Canny (Tl={tl} Th={th})', f'Circle200 Canny (Tl={tl} Th={th})']
plt.close('all')
view1d1 = viewer(image_list, title=title_list, subplots=(2,2), joint_zoom=True, widgets=True)

We will now experiment with setting the threshold values. In the following three cells, we pass a new GrayFig image through the steps you just coded. Test your understanding by answering the following MCQ.

In [ ]:
%use sos
# Apply Gaussian blurring to grayfig image
img_grayfig_blur = smooth_gaussian(img_grayfig)
# Compute gradient features of the grayfig image
img_grayfig_magphase = np.array(compute_gradient_features(img_grayfig_blur))

In [ ]:
%use javascript
%get img_grayfig_magphase
%put img_grayfig_nms 
%get tl th
%put img_grayfig_tlh
// Apply the non-maximum suppression to the gradient features of the grayfig image
var img_grayfig_nms = applyNonMaximumSuppressionInterpolation(
    new Image(img_grayfig_magphase[0]), new Image(img_grayfig_magphase[2]), new Image(img_grayfig_magphase[3]) 
).toArray();
// Perform hysteresis edge tracking on the nms output of the grayfig image
var img_grayfig_tlh = thresholdEdgesHysteresis(new Image(img_grayfig_nms), 10, 20).toArray();

In [ ]:
%use sos
# Convert JS arrays to numpy
img_grayfig_nms = np.array(img_grayfig_nms) 
img_grayfig_tlh = np.array(img_grayfig_tlh) 
# Display the different stages of the algorithm 
image_list = [img_grayfig, img_grayfig_blur, img_grayfig_nms, img_grayfig_tlh]
title_list = ['GrayFig Original', 'GrayFig Blurred', 'GrayFig NMS', f'GrayFig Canny (Tl={10} Th={20})']
plt.close('all')
view1d2 = viewer(image_list, title=title_list, subplots=(2,2), joint_zoom=True, widgets=True)

### Multiple Choice Question

* Q1: Which pair of thresholds `tl`, `th` would allow you to suppress the cross inside the inner (black) circle while retaining the rest of the lines on the outside ? Try to guess from the pixel values and build an intuition :-)

    1. 100, 130
    2. 60, 100
    3. 40, 70
    4. 50, 200
    
In the next cell, modify the variable `answer` to reflect your answer. The following cell is for you to check that your answer is in the valid range.

In [ ]:
%use sos

# Modify these variables
answer = None

# YOUR CODE HERE

In [ ]:
%use sos

# Sanity check
if not answer in [1, 2, 3, 4]:
    print('WARNING!\nChoose one of 1, 2, 3 or 4.')

Congratulations! You have now implemented all the building blocks of the Canny edge detector. It's time to try it out and compare your edges to the ones of OpenCV.

## 1.E. Comparison with OpenCV

[Back to table of contents](#ToC_2_Feature_detection)

In the next cell let us first define our comparison utility function. It will return an image where <b><i>red</i> pixels are edges your implementation has but OpenCV doesn't and <i>green</i> are pixels your implementation doesn't have but OpenCV does</b>.

In [ ]:
%use sos
# Function that highlights the difference between two images
def compare_difference(mystack, opencv, print_diff=False):
    # unpack stack, variables could come useful for some during debug
    image, blur, mag, phase, gradx, grady, nms, tlh = mystack
    # label differences
    diff_img = cv.normalize(cv.cvtColor(np.float32(opencv), code=cv.COLOR_GRAY2RGB), 
                            None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)
    diff_img[opencv > tlh] = [0, 255, 0] # GREEN: opencv has this pixel
    diff_img[opencv < tlh] = [255, 0, 0] # RED: opencv doesn't have this pixel
    # compute similarity
    diff_str = f'Original = OpenCV @ {100*(1-np.mean(tlh != opencv)):.3f}%'

    return diff_img, diff_str

Now let's compare your implementation to what OpenCV returns as edges for the image Object. First we define the parameters and apply the workflow that you have coded so far. Run the next two cells to do so.

In [ ]:
%use sos
# Set the parameters
sigma = 1.2
ksize = 3
tl, th = 10, 20
# Perform Gaussian blurring
img_object_blur = smooth_gaussian(img_object, sigma=sigma)
# Compute the gradient features for the JS function
img_object_magphase = np.array(compute_gradient_features(img_object_blur, ksize=ksize))

In [ ]:
%use javascript
%get img_object_magphase
%put img_object_nms
%get tl th
%put img_object_tlh
// Apply non-maximum suppression 
var img_object_nms = applyNonMaximumSuppressionInterpolation(
    new Image(img_object_magphase[0]), new Image(img_object_magphase[2]), new Image(img_object_magphase[3]) 
).toArray();
var img_object_tlh = thresholdEdgesHysteresis(new Image(img_object_nms), tl, th).toArray();

In the next cell we use [`cv.Canny(src, threshold1, threshold2, apertureSize, L2gradient)`](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de) to perform pure Python edge detection on `img_object`, and assign the value to `img_object_cv`. The cell below the next one will display both results, so you can compare them.

In [ ]:
%use sos
# Perform Canny edge detection in Python using cv.Canny
img_object_cv = None
# YOUR CODE HERE

In [ ]:
%use sos
# Convert JS arrays to numpy
img_object_nms = np.array(img_object_nms) 
img_object_tlh = np.array(img_object_tlh)
# Gett the difference between the two results
diff = compare_difference([img_object, img_object_blur, *img_object_magphase, img_object_nms, img_object_tlh], img_object_cv)
# Display the results
image_list = [img_object, img_object_tlh, img_object_magphase[0], img_object_cv, img_object_magphase[1], diff[0]]
title_list = ['Image Object', 'Original Canny', 'Magnitude', 'OpenCV Canny', 'Phase', diff[1]]
plt.close('all')
view1f = viewer(image_list, title=title_list, subplots=(3,2), widgets=True, joint_zoom=True)

You may be disappointed, don't worry, no points are involved in this one. In fact, no matter how you called that last function, the outputs are different. We just wanted you to open up the documentation.

After digging through OpenCV's implementation we can tell you why you see what you see. Going stage by stage, these are the differences :
<ol>     
    <li>For noise reduction, <b>OpenCV does not apply gaussian smoothing on the image</b>.
        <ul>
            <li>They rely on the inherent smoothing of the sobel operator. This explains why they have the apertureSize parameter which you have ignored so far. One can increase it for more blurring effect.</li>
            <li>We will provide OpenCV our blurred image directly</li>
        </ul>
    </li>
    <br>
    <li>For magnitude computation, OpenCV uses L1-norm to compute the magnitude, by default. We assume this is to save some extra computations at little performance cost.</li>
    <br>
    <li>For non-maximum suppression <b>OpenCV does not use interpolation</b>. 
        <ul>
            <li>Instead, they round the phase and take the nearest pixel in that direction. Since they do this, they do not even bother computing the phase and use mathemagic tricks to classify the direction of interest directly. However, this implies they <i>cheat</i> on the exactness of the maximum and we will show you how. It has to do with the <i>strict</i> vs <i>or equal</i> comparisons of neighbouring pixels.</li>
            <li>This also allows for some slackness for OpenCV to detect an edge where our implementation requires further blurring to ensure a maximum.</li>
            <li>If you really zoom in, you will notice the OpenCV implementation is asymmetric (keep in mind the original image might be asymmetric).</li>
        </ul>
    </li>
    <br>
    <li>For hysteresis thresholding, let's just say our algorithm is a bit more sensible to these labs but overall this stage's outputs are identical.</li>
    <br>
    <li>In general, we'd like to bring your attention to the matter of boundary conditions. We have chosen to use mirror boundaries but this is not the case for OpenCV. This allows them to classify more boundary pixels as edges.</li>
    <li>Finally, the fact that <b>OpenCV only accepts <code>uint8</code> input images</b> leads to numerical differences from the very start.
</ol>

That presents quite a few reasons and any combination of the above could explain why the images differ. We will go through them in the second half of this first part but we've spared you most of the trouble already. You might want to skim through their [code on github](https://github.com/opencv/opencv/blob/4.x/modules/imgproc/src/opencl/canny.cl), the implementation is super efficient.

<a id="last_part_comp_opencv"></a>
Using the following four cells, we invite you to visualize the differences on the images we introduced at the beginning of this notebook and experiment with some parameters. In particular, why not try images 3, 4 and 10 (which we will reuse) or explore the L1 vs L2 difference, for example? We apologize for not being able to provide interactive widgets to make this less tedious.

In [ ]:
%use sos

# Choose 1 out of 11 images [0..10]
img = images_list[-1]
# Set your parameters (ignore ksize and borderType)
sigma = 2
ksize = 3
tl, th = 10, 20
l2grad = True

# Apply Gaussian smoothing
blur = smooth_gaussian(img, sigma=sigma)
# Compute the gradient features
magphase = np.array(compute_gradient_features(blur))

In [ ]:
%use javascript
%get magphase
%put nms 
%get tl th
%put tlh
// Apply the non-maximum suppression to the gradient features
var nms = applyNonMaximumSuppressionInterpolation(
    new Image(magphase[0]), new Image(magphase[2]), new Image(magphase[3]) 
).toArray();
// Perform hysteresis edge tracking on the nms output
var tlh = thresholdEdgesHysteresis(new Image(nms), tl, th).toArray();

In [ ]:
%use sos
# Convert JS arrays to numpy
nms = np.array(nms) 
tlh = np.array(tlh)

# Perform numpy Canny edge detection
opencv = cv.Canny(np.uint8(img), threshold1=tl, threshold2=th, apertureSize=ksize, L2gradient=l2grad)

# Get the differences between the two versions
diff = compare_difference([img, blur, *magphase, nms, tlh], opencv)
# Display the differences
image_list = [img, tlh, magphase[1], opencv, nms, diff[0]]
title_list = ['Original', 'My Canny', 'Phase', 'OpenCV', 'NMS', diff[1]]
plt.close('all')
view1e2 = viewer(image_list, title=title_list, subplots=(3,2), widgets=True, joint_zoom=True)

## 1.F. The switch to OpenCV (2 points)

[Back to table of contents](#ToC_2_Feature_detection)

As we have presented in the previous section the differences our implementation has with OpenCV, we will now make the adjustments necessary to match the implementation exactly. You will then be able to better tell if you made a mistake in your above implementation.

Since we already set you up with easy ways to modify what needs to be changed the first time around, let's just reimplement the non-maximum suppression stage.

Again, we will ask that you to code in Javascript to get a lower level understanding of how this works. Your task, **for 2 points**, is to implement the function `applyNonMaximumSuppressionRound45deg`.

In this function, you will examine the gradient magnitude value with its 8-connected neighbors:

<center><img alt="8-connected neighbors" src="images/8-connected.png" width="300"></center>

whether its value is greater than its two neighbours along the horizontal, vertical, diagonal and antidiagonal directions. If this is the case, we will keep it, otherwise we will discard it. 

Specifically, greater means **greater or equal** for the neighboring pixels directly below or to the right and **strictly greater** for the others (check the visualization below): 

<ul>
    <li>$(x-1, y) < (x, y)  \ge  (x+1, y)$</li>
    <li>$(x, y-1) < (x, y)  \ge  (x, y+1)$</li>
    <li>$(x-1, y-1) < (x, y)  >  (x+1, y+1)$</li>
    <li>$(x-1, y+1) < (x, y)  >  (x+1, y-1)$</li>
    </ul>

Furthermore, use **zero padding**. You may do so directly through `getPixel(x, y, padding='zero')`.



<div class="alert alert-info">
    
**Note:** These asymmetric comparisons favor detection in the bottom right direction whereas our implementation is isotropic. The strict comparisons in the diagonals aim to compensate for the greater distance, which our comparisons based on uniform gradient vectors does not require. Although quite imprecise at first, from a pure image processing point of view, the choices are very well balanced and the differences remain subtle in most applications.
</div>

The function and its parameters are defined as:

<code>applyNonMaximumSuppressionRound45deg(image, phase)</code>
<ul>
    <li><code>image</code> : An <code>Image</code> object that contains the magnitude of the gradient at each pixel</li>
    <li><code>phase</code> : An <code>Image</code> object that contains the direction of the gradient at each pixel</li>
</ul>

In [ ]:
%use javascript

function applyNonMaximumSuppressionRound45deg(image, phase) {
    var out = new Image(image.shape());
    
    // YOUR CODE HERE
    
    return out;
}

Let's perform a quick sanity check using the gradient features of a randomized image computed using `ksize=3`. This should highlight the differences between `Interpolation` and `Round45deg` non-maxima suppressions. The interpolated version should give better boudaries for the middle three squares, whereas the rounding version should detect the bottom left square due to the adjusted boundary conditions (you might want to go back to the interpolation if this doesn't feel right). Run the following three cells.

In [ ]:
%use sos
# Create the test image
img_rand = np.zeros((7,7))
img_rand[1,5] = img_rand[2,2] = img_rand[3,3] = img_rand[4,4] = img_rand[6,0] = 1
img_rand_magphase = np.array(compute_gradient_features(img_rand, ksize=3))

In [ ]:
%use javascript
%get img_rand_magphase
%put img_rand_nms_i img_rand_nms_r
// apply the round version of nms on the test image
var img_rand_nms_r = applyNonMaximumSuppressionRound45deg(
    new Image(img_rand_magphase[0]), new Image(img_rand_magphase[1]) 
).toArray();
// apply the interpolation version of nms on the test image
var img_rand_nms_i = applyNonMaximumSuppressionInterpolation(
    new Image(img_rand_magphase[0]), new Image(img_rand_magphase[2]), new Image(img_rand_magphase[3]) 
).toArray();

In [ ]:
%use sos
# Convert JS arrays to numpy
img_rand_nms_i = np.array(img_rand_nms_i)
img_rand_nms_r = np.array(img_rand_nms_r)
# Visualize the outputs
image_list = [img_rand, np.zeros_like(img_rand), img_rand_magphase[0], img_rand_magphase[1], img_rand_nms_i, img_rand_nms_r]
title_list = ['Randomised Image', 'Intentionally Blank', 'Magnitude', 'Gradient Direction', 'NMS with Interpolation', 'NMS with Round to 45deg']
plt.close('all')
view = viewer(image_list, title=title_list, subplots=(3,2), joint_zoom=True, widgets=True)
# Sanity check, counting the pixel values of the output
if int(np.sum(impulse_nms)) != 93:
    print('WARNING!\nYour function did not suppress the right pixels or you modified their values.')
else:
    print('Congratulations! Your non-maximum suppression passed the sanity check.\nRemember that this is not a guarantee that everything is correct.')

Now that perform the same orientation rounding as OpenCV, the last thing to change is to call your `compute_gradient_features` with `sigma=None`, to not apply any blurring, and with `borderType=cv.BORDER_REPLICATE`, so that we're using the same boundary condition as OpenCV. Again, remember that in image processing, border conditions do make a difference. Now that we have established fair play, let's compare your implementation to the one of OpenCV using the Grayfig image.

In [ ]:
%use sos
# Define the parameters
sigma = None  # explicitely not used
ksize = 3
tl, th = 10, 20
# Convert the test image to uint8 to match OpenCV
img_canny = np.uint8(img_grayfig)
# Compute the gradient features
img_grayfig_magphase_r = np.array(compute_gradient_features(img_grayfig, ksize=ksize, borderType=cv.BORDER_REPLICATE))

In [ ]:
%use javascript
%get img_grayfig_magphase_r
%put img_grayfig_nms_r
%get tl th
%put img_grayfig_tlh_r
// apply the rounding nms
var img_grayfig_nms_r = applyNonMaximumSuppressionRound45deg(new Image(img_grayfig_magphase_r[0]),
                                                             new Image(img_grayfig_magphase_r[1]) ).toArray();
// apply hysteresis edge tracking
var img_grayfig_tlh_r = thresholdEdgesHysteresis(new Image(img_grayfig_nms_r), tl, th).toArray();

In [ ]:
%use sos
# Convert JS arrays to numpy
img_grayfig_nms_r = np.array(img_grayfig_nms_r) 
img_grayfig_tlh_r = np.array(img_grayfig_tlh_r)
# Apply the OpenCV Canny edge detection
img_grayfig_cv_r = cv.Canny(img_canny, threshold1=tl, threshold2=th, apertureSize=ksize, L2gradient=True)
# Get the difference between the two versions
diff_r = compare_difference([img_grayfig, None, *img_grayfig_magphase_r, img_grayfig_nms_r, img_grayfig_tlh_r], img_grayfig_cv_r)
# Visualize the results
image_list=[img_grayfig_tlh_r, img_grayfig_cv_r, diff_r[0]]
title_list=['Original', 'OpenCV', diff_r[1]]
plt.close('all')
view1f = viewer(image_list, title=title_list, subplots=(1,3), widgets=True, joint_zoom=True)

Since you took the time to code both implementations, you might want to observe the differences of using the interpolated vs rounded versions of non-maximum suppression. We invite you to go back to the [last part of *Comparison with OpenCV*](#last_part_comp_opencv) and change `applyNonMaximumSuppressionInterpolation` to `applyNonMaximumSuppressionRound45deg`. You may need to zoom in to appreciate the differences, and/or better understand why OpenCV decided not to interpolate.

# 2. Original ridge detector (3 points)
[Back to table of contents](#ToC_2_Feature_detection)

The original ridge detector we will implement in this section will make use of the building blocks of the Canny edge detector. We will be able to reuse most of the Canny edge detector's pipeline. 

The ridge detection algorithm is as follows.
1. Noise reduction
2. Figure of merit and eigenvector orientation based on the Hessian matrix
3. Non-maximum suppression
4. Hysteresis thresholding

As portrayed by the following diagram, you implemented most of these already, only the second step will get replaced by features of the Hessian (instead of the gradient). 

[<img src="images/ridge_detector_schematic.png" width="800"/>](images/ridge_detector_schematic.png)

## 2.A. Motivation

[Back to table of contents](#ToC_2_Feature_detection)

Go ahead and run the next cell to visualise the limitations of the edge detector. If you observe attentively the output of the edge detector, you will see that for each line we get two edges. In some applications it can be useful to detect only the ridge line (vessels in the next figure). This is what we would like our ridge detector to do, with very little modifications to our edge detection pipeline.

In [ ]:
%use sos
# Visualize the fungus image
image_list = [img_fundus, cv.Canny(img_fundus, threshold1=70, threshold2=120, apertureSize=3, L2gradient=True)]
title_list = ['Fundus Original','Fundus Edges']
plt.close('all')
view2a = viewer(image_list, title=title_list, subplots=(1,2), widgets=True, joint_zoom=True)

## 2.B. Implementation (3 points)

[Back to table of contents](#ToC_2_Feature_detection)


Following the explanation given in the introduction, we will now guide you through the computation of the [Hessian matrix](https://en.wikipedia.org/wiki/Hessian_matrix) features used by the ridge detection algorithm.

In the next cell, we define a utility function that will return the eigenvector associated to the provided eigenvalue and Hessian matrix elements. The inputs and outputs are all images so you will need to call this function only once.

In the following cell, **for 3 points**, implement `compute_hessian_features(image, ksize, borderType)`. Just as for the gradient features, use the OpenCV Sobel filter [`cv.Sobel`](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gacea54f142e81b6758cb6f375ce782c8d) to compute the elements of the Hessian matrix and follow the comments in the code to compute the desired outputs.

The function and its input parameters is defined as:

`compute_hessian_features(image, ksize, borderType)`
* `image` : The input image, from where we want to compute the features
* `ksize` : The size of the Sobel filter
* `borderType` : The boundary conditions to use for the Sobel filter

The function returns:
* `merit` : An image containing the **figure of merit** (see hints) for each pixel
* `orientation` : An image containing the orientation of the **eigenvector associated to the minimum eigenvalue** for each pixel
* `vminX` : An image containing the x-component of the eigenvector associated to the minimum eigenvalue for each pixel
* `vminY` : An image containing the y-component of the eigenvector associated to the minimum eigenvalue for each pixel

<div class="alert alert-warning">
    <b>Important:</b> Use the utility function <code>get_hessian_eigenvector(eigenvalue, hessianXX, hessianXY, hessianYY)</code> to get the x- and y-components of the eigenvector associated to a given eigenvalue and the Hessian matrix elements.
</div>

<div class="alert alert-info">
<b>Hints:</b> 
<ul>
    <li><code>cv.Sobel</code>: set <code>ddepth=cv.CV_64F</code> since the gradient can be a negative floating point number.</li>
    <li><code>cv.Sobel</code>: You can set the order of the <code>x</code>- and/or <code>y</code>-derivative you want to generatate by setting <code>dx=order_x</code> and <code>dy=order_y</code>.</li>
    <li>The hessian matrix is symmetric.</li>
    <li>The figure of merit is defined as $fom=\sqrt{|\lambda_{min}| |\lambda_{min} - \lambda_{max}|}$, where $\lambda_{min}$ and $\lambda_{max}$ are the minimum and maximum eigenvalues of the Hessian matrix respectively.</li>
    <li><code><a href="https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html" style="color:black">np.arctan2</a></code>: could help you get the correct orientation in all four quadrants.</li>
    <li>To ensure that you define the correct parameter in a function call always specify the name of the parameter.</li>
</ul>
</div>

In [ ]:
%use sos
# Function that returns the eigenvector associated to a given eigenvalue and the Hessian matrix elements
def get_hessian_eigenvector(eigenvalue, hessianXX, hessianXY, hessianYY):
    # compute a normalized eigenvector of [[Hxx, Hxy],[Hyx, Hyy]] associated to lmin: [vminX, vminY].T
    vx = np.zeros(hessianXX.shape)
    vy = np.zeros(hessianXX.shape)
    norm = np.ones(hessianXX.shape)   
    
    # sometimes the norm of the computed eigenvector is nil and we can take the other eigenvector to compensate
    # ignore situations where both are nil simultaneously as the figure of merit is irrelevant (0)
    # get first eigenvectors (values default to zero by init)
    v1 = np.array([hessianXY, eigenvalue - hessianXX]) 
    norm1 = np.sqrt(v1[0]**2 + v1[1]**2)
    vx[norm1 > 0] = v1[0][norm1 > 0] / norm1[norm1 > 0]
    vy[norm1 > 0] = v1[1][norm1 > 0] / norm1[norm1 > 0]
    # override or complete first eigenvectors with second eigenvectors
    v2 = np.array([eigenvalue - hessianYY, hessianXY])
    norm2 = np.sqrt(v2[0]**2 + v2[1]**2)
    vx[norm2 > 0] = v2[0][norm2 > 0] / norm2[norm2 > 0]
    vy[norm2 > 0] = v2[1][norm2 > 0] / norm2[norm2 > 0]
    
    return vx, vy

In [ ]:
%use sos
# Function that computes the Hessian features of a given input image
def compute_hessian_features(image, ksize=3, borderType=cv.BORDER_REFLECT):
    merit = np.zeros(image.shape)
    orientation = np.zeros(image.shape)
    vminX = np.zeros(image.shape)
    vminY = np.zeros(image.shape)

    # Compute the Hessian matrix elements (xx, xy, and yy) using the cv.Sobel function with different orders of dx and dy
    # YOUR CODE HERE
    
    # Solve the characteristic polynomial equation to determine the minimum and maximum eigenvalues
    # YOUR CODE HERE
    
    # Define the figure of merit and orientation of eigenvector for non-maxima suppression
    # Make sure to use `get_hessian_eigenvector` (we cannot check all variations of possible answers)
    # YOUR CODE HERE
    
    return merit, orientation, vminX, vminY

Let's perform a quick sanity check and see if we can extract the middle centerline of Cross200 given a sufficiently large `sigma=3`. Run the following cell to visualize the output and differences between the magnitude and the figure of merit.

In [ ]:
%use sos
# Apply Gaussian blurring to the cross image
cross_blur = smooth_gaussian(img_cross200, sigma=3)
# Compute the Hessian features
cross_ridge_feats = np.array(compute_hessian_features(cross_blur))
# Compute the gradient features
cross_edges_feats = np.array(compute_gradient_features(cross_blur))
# Visualize the blurred image, gradient magnitude and the figure of merit
image_list = [cross_blur, cross_edges_feats[0], cross_ridge_feats[0]]
title_list = ['Blurred Cross200', 'Gradient magnitude', 'Figure of merit']
plt.close('all')
view2b1 = viewer(image_list, title=title_list, subplots=(1,3), widgets=True, joint_zoom=True)

# Sanity checks
error_check = False

# Figure of merit values expected to be positive
if not np.all(cross_ridge_feats[0] >= 0):
    print('WARNING!\nYour figure of merit values should all be positive.')
    error_check = True

# The cross centerline should be bright
if np.mean(cross_ridge_feats[0][100,:] + cross_ridge_feats[0][:,100]) < 35 :
    print('WARNING!\nSeems like your figure of merit has very low merit at the centerline, this is the opposite of what we aim for here.')
    error_check = True

# Normalised eigenvector norms are within [-1, 1]
if np.any(np.abs(cross_ridge_feats[2]) > 1) or np.any(np.abs(cross_ridge_feats[3]) > 1):
    print('WARNING!\nYour eigenvector norms should be in the range [-1, 1].') 
    error_check = True
    
if error_check:
    print('Make sure your calculation of the eigenvalues are correct. The other steps are similar to Part 1.B.')
else :
    print('Congratulations! Your hessian features passed the sanity check.\nRemember that this is not a guarantee that everything is correct.')

To close up this exercise, let's get back to our Fundus example. Running the next three cells will show you the output at different steps of your ridge detection pipeline. Given thresholds `tl=10` and `th=35` you should be able to distinguish the major vessels as single ridge lines.

In [ ]:
%use sos
# Apply Gaussian smoothing and compute Hessian features of the fundus image
hessian_blur = smooth_gaussian(img_fundus, sigma=1)
hessian_feats = np.array(compute_hessian_features(hessian_blur))

In [ ]:
%use javascript
%get hessian_feats
%put hessian_nms hessian_tlh
// Apply the non-maximum suppression with interpolation
var hessian_nms = applyNonMaximumSuppressionInterpolation(
    new Image(hessian_feats[0]), new Image(hessian_feats[2]), new Image(hessian_feats[3])
).toArray()
// Perform hysteresis edge tracking
var hessian_tlh = thresholdEdgesHysteresis(new Image(hessian_nms), 10, 35).toArray(); 

In [ ]:
%use sos
# Convert JS arrays to numpy
hessian_nms = np.array(hessian_nms)
hessian_tlh = np.array(hessian_tlh)
# Visualize the different steps in the pipeline
image_list = [img_fundus, hessian_blur, *hessian_feats[:2], hessian_nms, hessian_tlh]
title_list = ['Original', 'Blurred', 'Figure of Merit', 'Eigenvector Orientation', 'NMS', 'Ridges']
plt.close('all')
view2b2 = viewer(image_list, title=title_list, subplots=(3,2), widgets=True, joint_zoom=True)

This algorithm you just implemented has no equivalent in OpenCV. Other libraries such as scikit provide [ridge operators](https://scikit-image.org/docs/stable/auto_examples/edges/plot_ridge_filter.html). We invite you to play with them in the following cell, and observe the differences. We hope you will agree that your original ridge detector does a better job than this library, if you spend the time to tune your output, that is :) 

Also, some of these filters are named after people who worked for our lab!

In [ ]:
%use sos
# Import the ridge filters
from skimage.filters import meijering, sato, frangi, hessian

# Apply different ridge filters on the fundus image
test_img = images_list[2]
m = meijering(test_img)
s = sato(test_img)
f = frangi(test_img)
h = hessian(test_img)

# Display them
plt.close('all')
view2b3 = viewer([m, s, f, h], title=['meijering', 'sato', 'frangi', 'hessian'], subplots=(2,2), widgets=True, joint_zoom=True)

<div class="alert alert-success">
    <b>Congratulations on finishing the Feature detection lab !!</b> Hopefully you now understand the differences between edges and ridges, as well as their different use cases. Also, remember OpenCV is made to be fast and practical, sometimes at the expense of the purity we seek in this image processing course.
</div>

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/course/view.php?id=463">Moodle</a>, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *2_Feature_detection.ipynb*,
* Name the `zip` file: *Feature_detection_lab.zip*.